In [1]:
#Import Libraries
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bs4
import re
from googlesearch import search  
import warnings
warnings.filterwarnings('ignore')

/home/harikishan/anaconda3/envs/Hari/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [3]:
old_links = []
link_count = 1
with open("old_links.txt", "r") as old_links_file:
    for line in old_links_file:
        old_links.append(line.strip())
        link_count += 1
old_links_file.close()

In [10]:
query = "Digital loans challenges and solutions to overcome in United states, European countries"
links = []
for url in search(query, num_results=10, sleep_interval=2,advanced=False):
    print(url)
    if url not in old_links:
        links.append(url)
        old_links.append(url)
        print("new link found")
    else:
        print("exists in old")
        
with open("old_links.txt", "w") as old_links_file1:
    for line in old_links:
        old_links_file1.write(line+'\n')
old_links_file1.close()

https://www.eesc.europa.eu/sites/default/files/files/qe-01-19-295-en-n.pdf
new link found
https://www.oecd.org/competition/digital-disruption-in-banking-and-its-impact-on-competition-2020.pdf
new link found
https://www.bis.org/publ/bppdf/bispap117.pdf
new link found
https://www.imf.org/en/About/Factsheets/IMF-Lending
new link found
https://www.whitehouse.gov/briefing-room/statements-releases/2023/05/31/u-s-eu-joint-statement-of-the-trade-and-technology-council-2/
new link found
https://www.gatesfoundation.org/our-work/programs/global-growth-and-opportunity/financial-services-for-the-poor
new link found
https://www.worldbank.org/en/who-we-are/ibrd
new link found
https://www.adb.org/sites/default/files/publication/182532/adbi-wp564.pdf
new link found
https://www.un.org/sustainabledevelopment/wp-content/uploads/2022/03/2021-Report.pdf
exists in old
https://www.mckinsey.com/industries/financial-services/our-insights/global-banking-annual-review
new link found
https://espas.eu/files/espas_f

In [5]:
#links
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
import spacy

sp = spacy.load("en_core_web_sm")

all_stopwords = sp.Defaults.stop_words
#all_stopwords.add('<','title>','</title>','<p class="_21c54">','</p>')
all_stopwords = {'<','/title','title=','>','/','p','class=','_21c54',
                  '/p','/a','href=','#','/sup','[',']', 'cite_ref-',
                  'cite_note-','sup','br/','https://','id=',"''",'<p>',
                  '<p class=','<p align="center"','class=','<a href=','='}

#url = 'https://www.imf.org/-/media/Files/Publications/WP/2021/English/wpiea2021090-print-pdf.ashx'

2023-06-07 15:33:58.165179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 15:33:58.383254: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 15:33:58.423036: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/gurobi1000/linux64/lib:/opt/gurobi1000/linux64/lib
2023-06-07 15:33:58.423055

In [8]:
df=pd.DataFrame(columns=['Title','Content'])
for url in links: 
    print("Parsing: ",url)
    cont=""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")
    title=str(soup.find('title'))
    content = str(soup.findAll('p'))
    text_tokens = word_tokenize(content)
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]
    tle=""
    for t in tokens_without_sw:
        tle += ' ' + t
    #print(tle[:1000])
    title = title[7:len(title)-8]
    data={'Title':title,'Content':tle}
    df= df.append(data,ignore_index = True)

In [9]:
for i in range(len(links)):
    print(i, df['Title'].iloc[i], '(Content)' + str(df['Content'].iloc[i])[:20])

0  (Content) % PDF-1.4 % ���� 13
1 Financial assistance instruments | European Stability Mechanism (Content) , member__position 
2  (Content) % PDF-1.6 % ���� 40
3  (Content) % PDF-1.6 % ���� 35
4 You’ve Received a Message: Your Loan Has Been Approved (Content) c02v2-credit , styl
5  (Content) % PDF-1.6 % ���� 18
6 The IMF: The World’s Controversial Financial Firefighter | Council on Foreign Relations (Content) card-article__topic
7 ERROR: The request could not be satisfied (Content)
8  (Content) % PDF-1.5 % ���� 10
9  (Content) % PDF-1.5 % ���� 1 


In [ ]:
df

In [259]:
import math
import openai

openai.api_key = '###### USE YOUR API KEY #######'

def call_chatGPT(query):
    messages = [{"role": "user", "content": query}]
    chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    reply = chat.choices[0].message.content
    return reply

def summarize_ChatGPT(texts):
    summarized_text = ""
    max_tokens = 4000
    print("Input size = ",len(texts))
    for i in range(math.ceil(len(texts)/max_tokens)):
        text_chunk = texts[i*max_tokens:(i*max_tokens)+max_tokens]
        print("{:.2f}".format((i/math.ceil(len(texts)/max_tokens))*100),'%')
        query = "Summarize and list the challenges and its respective solutions to digital lending with headings: " + text_chunk
        summarized_text = summarized_text + '\n***START***\n' + call_chatGPT(query) + '\n***END***\n' 
        time.sleep(20)
    print("Output size = ",len(summarized_text))
    return summarized_text

In [260]:
#df.describe()['Title'].iloc[0]
type(df['Title'].count())

numpy.int64

In [262]:
text_file = open("test.txt", "a")
for i in range(1,df['Title'].count()):
    summary = summarize_ChatGPT(df['Content'].iloc[i])
    print('Link ',i+1)
    #summary = "Hi"
    texts = '\n===================================================================================================================================================================\n'\
            + 'Query: ' + query + '\n'\
            + str(link_count) + ' Title: ' + df['Title'].iloc[i] \
            + '\nReference: ' + links[i] \
            + '\nContent: \n'+ summary \
            + '\n====================================================================================================================================================================\n\n\n'
    link_count += 1
    text_file.write(texts)
text_file.close()

Input size =  10151
0.00 %
33.33 %
66.67 %
Output size =  2603
Link  2
Input size =  8524
0.00 %
33.33 %
66.67 %
Output size =  3853
Link  3
Input size =  403
0.00 %
Output size =  1701
Link  4
Input size =  17339
0.00 %
20.00 %
40.00 %
60.00 %
80.00 %
Output size =  8029
Link  5
Input size =  15861
0.00 %
25.00 %
50.00 %
75.00 %
Output size =  5090
Link  6
Input size =  5895
0.00 %
50.00 %
Output size =  1548
Link  7
Input size =  14524
0.00 %
25.00 %
50.00 %
75.00 %
Output size =  5187
Link  8
Input size =  7871
0.00 %
50.00 %
Output size =  2592
Link  9
Input size =  26163
0.00 %
14.29 %
28.57 %
42.86 %
57.14 %
71.43 %
85.71 %
Output size =  6827
Link  10
Input size =  8640
0.00 %
33.33 %
66.67 %
Output size =  5422
Link  11
